In [1]:
import os
import yaml

# os.chdir('//share.ahc.ufl.edu/share$/DSS/IDR_Projects/COVID-19/OMOP/new_pipeline')
config_path = 'Y:/IDR_Projects/COVID-19/OMOP/new_pipeline/config.yml'
# print(os.getcwd())

import pandas as pd
from omop_etl.load import Loader
# from .omop_etl.stage import STAGE

from pydma.utils import find

In [5]:
omop = Loader(config_path)
STAGE = omop.stage

# Row counts

In [2]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [6]:
loaded = omop.config.load

result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                count = omop.row_count(stg_name, schema='stage')
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            count = omop.row_count(stg_name, schema='stage')
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Stage_RC'])
# table_counts.head(50)

In [6]:
# table_counts

In [7]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['Preload_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='preload') if t not in ('death','person','visit_occurrence') else 0)
count_diff['Load_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['DeId_RC'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))

In [10]:
# t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()

In [8]:
print(count_diff)

                  Table   Stage_RC  Preload_RC     Load_RC     DeId_RC
0  condition_occurrence  173518137   177823426   146150593   146150593
1                 death      30650           0       30650       30650
2         drug_exposure  190338221   190870092   190870762   190870762
3           measurement  895874560  1045335172  1042696122  1042696122
4           observation  166369054   166568652   235179838   235179838
5                person     542101           0      542100      542100
6  procedure_occurrence   64499868    64501094    29273918    29273918
7      visit_occurrence   55281080           0    55281069    55281080


# Count concept ids

# Date range

# Test for bad values
- bp - charindex('/',(bp),1) > 0
- res_resp_mech - leng of value_source_value <=50